In [0]:
pip install h2o

In [0]:
pip install fuzzywuzzy

In [0]:
pip install plotnine

In [0]:
pip install --upgrade pip

In [0]:
pip install xgboost

In [0]:
pip install shap

In [0]:

pip install catboost

In [0]:
pip install lightgbm

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,current_date, expr,mean,col, sum as spark_sum,count,avg
from pyspark.sql.types import DoubleType,IntegerType,StringType,NumericType
from pyspark.ml.feature import OneHotEncoder, StringIndexer, Imputer, VectorAssembler
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
#from h2o.automl import H2OAutoML
from sklearn.metrics import mean_squared_error, r2_score
#import h2o
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor

## Data Import

In [0]:
import requests

sharepoint_url = 'https://cmhcschl.sharepoint.com/:x:/s/FinancingOperation-Legacyteam-SupportandInnovationteam/EZA0yVuxjzJCoqM4DJvoohkBJVCYqbPQ6x2jExfTbuCfug?e=4%3AGfYMdr&at=9&CID=2C9A02A9-E602-46FA-B712-CB50AEDFEAB4&wdLOR=c7117F0A1-54E1-463B-AC5F-489B07B48987'

# Send a GET request to download the file
response = requests.get(sharepoint_url)

# Check if the request was successful
if response.status_code == 200:
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(response.content)

    # Now you can work with the DataFrame as needed
    print(df.head())
else:
    # Display an error message if the download failed
    print("Failed to download the Excel file.")


In [0]:
folder_path = "/Workspace/Users/zzhang@cmhc-schl.gc.ca/TimeStudy-2023Jun.xlsx"  # Update the folder path as per your directory structure
TS = pd.read_excel(folder_path,sheet_name = 'Advancing Data',header=2)

In [0]:
TS

## Data Preprocessing

In [0]:
TS_c = TS.fillna(0,inplace=False)
TS_c

In [0]:
TS_c.shape

In [0]:
#is there any categorical columns?
TS_c.info()

In [0]:
TS_c.duplicated().sum()

In [0]:
TS_mask = TS_c.duplicated()
if TS_mask.any():
    TS_c = TS_c[~TS_mask]
TS_c.duplicated().sum()

## Data Exploratory

In [0]:
TS_c.columns

In [0]:
for col in TS_c.columns:
    if TS_c[col].dtype == 'object':
        counts = TS_c[col].value_counts()
        print(f"Value Counts for '{col}' is :")
        print(counts)

In [0]:
# Calculate the sum of 'Total Minutes' for each staff in each collection period
staff_sum_df = TS_c.groupby(['Staff Name:', 'Collection Period:'])['Total Minutes:'].sum().reset_index()

# Convert the sum of 'Total Minutes' to hours by dividing by 60 and round to two decimal places
staff_sum_df['Total Minutes:'] = (staff_sum_df['Total Minutes:'] / 60).round(2)

# Calculate the overall average of 'Total Minutes' across all staff and all collection periods
overall_avg_minutes = TS_c.groupby('Collection Period:')['Total Minutes:'].mean() / 60  # Convert to hours and calculate mean

# Create the Plotly bar plot
fig = px.bar(staff_sum_df, x='Staff Name:', y='Total Minutes:', color='Staff Name:',
             color_continuous_scale='Viridis',
             labels={'Total Minutes:': 'Total Hours (Sum)', 'Staff Name:': 'Staff Name', 'Collection Period:': 'Month'},
             title='Total Hours per Staff by Collection Period',
             text='Total Minutes:',
             facet_row='Collection Period:')

# Add a line trace for the overall average
fig.add_shape(type='line',
              x0=-0.5, x1=len(staff_sum_df['Staff Name:']) - 0.5,
              y0=overall_avg_minutes, y1=overall_avg_minutes,
              line=dict(color='red', width=2))

# Customize the layout
fig.update_layout(xaxis_tickangle=-45, xaxis_title='Staff Name', yaxis_title='Total Hours', showlegend=False, height=800)

fig.show()

In [0]:
# Calculate the sum of 'Total Minutes' for each staff in each collection period
staff_sum_df = TS_c.groupby(["Program:", 'Collection Period:'])['Total Minutes:'].mean().reset_index()

# Convert the sum of 'Total Minutes' to hours by dividing by 60 and round to two decimal places
staff_sum_df['Average Minutes:'] = (staff_sum_df['Total Minutes:'] / 60).round(2)

# Create the Plotly bar plot
fig = px.bar(staff_sum_df, x='Program:', y='Total Minutes:', color='Program:',
             color_continuous_scale='Viridis',
             labels={'Total Minutes:': 'Total Hours (Sum)', 'Program:': 'Program', 'Collection Period:': 'Month'},
             title='Total Hours per Program by Collection Period',
             text='Total Minutes:',
             facet_row='Collection Period:')

# Add a line trace for the overall average
fig.add_shape(type='line',
              x0=-0.5, x1=len(staff_sum_df['Program:']) - 0.5,
              y0=overall_avg_minutes, y1=overall_avg_minutes,
              line=dict(color='red', width=2))

# Customize the layout
fig.update_layout(xaxis_tickangle=-45, xaxis_title='Program', yaxis_title='Total Hours', showlegend=False, height=800)

fig.show()

In [0]:
import plotly.express as px

# Calculate the sum of 'Total Minutes' for each staff in each program
staff_sum_df = TS_c.groupby(["Program:"])['Total Minutes:'].mean().reset_index()

# Convert the sum of 'Total Minutes' to hours by dividing by 60 and round to two decimal places
staff_sum_df['Total Hours (Sum)'] = (staff_sum_df['Total Minutes:'] / 60).round(2)

# Create the Plotly bar plot
fig = px.bar(staff_sum_df, x='Program:', y='Total Hours (Sum)', color='Program:',
             color_continuous_scale='Viridis',
             labels={'Total Hours (Sum)': 'Total Hours (Sum)', 'Program:': 'Program'},
             text='Total Minutes:',
             title='Total Hours Spent by Program')

# Customize the layout
fig.update_layout(xaxis_tickangle=-45, xaxis_title='Program', yaxis_title='Total Hours (Sum)', showlegend=False, height=600)

fig.show()

In [0]:
#base on time it takes for program, compare the average of each program and individual/ 
#compare the time they spent one advancing and the rest of their time

In [0]:
from plotnine import ggplot, aes, geom_histogram,facet_grid,theme,element_text,geom_text,ylab
print(ggplot(TS_c)+aes(x="Staff Name:")+ylab("Num of Advancing Sloven")+geom_histogram(binwidth=0.5)+facet_grid(facets='~Collection Period:')+theme(axis_text_x=element_text(angle=45, ha='right',size=6))+geom_text(aes(label='stat(count)'), stat='count', va='bottom', format_string="{:.0f}",size=5))
#make the pic larger
#distinguish the owner and creator

In [0]:
#Uniform values in "Stage" and 'Budget type" by lowercasing certain letters
# Convert the values in "Stage" column to lowercase
TS_c['Stage:'] = TS_c['Stage:'].str.lower()
TS_c['Budget Type:'] = TS_c['Budget Type:'].str.lower()
TS_c['Program:'] = TS_c['Program:'].str.lower()

In [0]:

TS_c['Stage:'].value_counts()
TS_c['Budget Type:'].value_counts()
TS_c['Program:'].value_counts()

In [0]:
# Count the occurrences of each "CRM File #" and create a new column "CRM File # Count"
TS_c['CRM File # Count'] = TS_c.groupby('CRM File #:')['CRM File #:'].transform('count')

# Set rows where "CRM File #" occurs less than 3 as "other"
TS_c.loc[TS_c['CRM File # Count'] < 15, 'CRM File #:'] = 'other'

# Drop the temporary "CRM File # Count" column if you don't need it anymore
TS_c.drop(columns=['CRM File # Count'], inplace=True)

In [0]:
TS_c['CRM File #:'].value_counts()

In [0]:
unique_values_count = TS_c['Program:'].nunique()
print("Number of unique values:", unique_values_count)

In [0]:
# Create dummy variables for the "Budget Type" column
dummy_budget_type = pd.get_dummies(TS_c['Budget Type:'])

# Keep both "contribution/forgiveable loan" and "repayable loan" columns
dummy_budget_type = dummy_budget_type[['both (contribution/forgiveable loan & repayable loan)', 'repayable loan']]

# Concatenate the dummy variables back to the original DataFrame
TS_c = pd.concat([TS_c, dummy_budget_type], axis=1)

# Drop the original "Budget Type" column
TS_c.drop('Budget Type:', axis=1, inplace=True)

In [0]:
pd_con = TS_c['Program:'].str.get_dummies().drop('federal lands initiative (fli)', axis=1)

In [0]:
pd_con = pd_con.add_prefix('Program:_')

In [0]:
TS_c_dropped = TS_c.drop(columns=['Program:'])

# Get dummies for the DataFrame after dropping the 'Program:' column
TS_f = pd.get_dummies(TS_c_dropped, drop_first=True)

In [0]:
combined_df = pd.concat([pd_con, TS_f], axis=1)

In [0]:
combined_df.columns

In [0]:
TS_f = combined_df

In [0]:
# Calculate the correlation matrix
corr_matrix = TS_f.corr()

# Set the threshold for high correlation (you can adjust this value as needed)
threshold = 0.5

# Initialize a list to store highly correlated variable pairs
highly_correlated_pairs = []

# Iterate through the correlation matrix and identify highly correlated variables
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        correlation = corr_matrix.iloc[i, j]
        if abs(correlation) > threshold:
            var1 = corr_matrix.columns[i]
            var2 = corr_matrix.columns[j]
            highly_correlated_pairs.append((var1, var2, correlation))

# Print the highly correlated variable pairs
if highly_correlated_pairs:
    print("Highly correlated variable pairs:")
    for pair in highly_correlated_pairs:
        var1, var2, correlation = pair
        print(f"{var1} and {var2}: {correlation:.2f}")
else:
    print("No highly correlated variables found.")
    #keep activity phase and program

In [0]:
# Drop the columns using a range of column names
TS_f.drop(columns=TS_f.loc[:, 'Time Record #1':'Time Record #10'], inplace=True)

In [0]:
TS_f.columns

In [0]:
TS_f= TS_f.drop(columns=['repayable loan'])

In [0]:
# Split your data into training and testing sets
y = TS_f['Total Minutes:']
X = TS_f.drop('Total Minutes:',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
X

In [0]:
X.columns

In [0]:
# Initialize H2O and start the H2O server
h2o.init()

# Convert the pandas DataFrames to H2O DataFrames
train_data = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
test_data = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))

# Specify the name of the target variable
target_variable = 'Total Minutes'

# Run AutoML regression
aml = H2OAutoML(max_runtime_secs=120)
aml.train(y=target_variable, training_frame=train_data)

# Get the best model from AutoML
best_model = aml.leader

y_test_pred = best_model.predict(test_data).as_data_frame().values.flatten()

y_test_pred = np.array(y_test_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Test Mean Squared Error: {test_mse:.2f}")
print(f"Test R-squared: {test_r2:.2f}")

# Visualize the comparison of forecast and actual values for both training and test sets
plt.figure(figsize=(10, 6))

plt.scatter(y_test, y_test_pred, color='orange', alpha=0.5, label='Test Data')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', linewidth=2)
plt.xlabel("Actual Values (Test)")
plt.ylabel("Predicted Values (Test)")

plt.legend()
plt.title("Comparison of Forecast and Actual Values")
plt.show()

In [0]:
from autosklearn.regression import AutoSklearnRegressor
# Create the AutoML regression model
automl_regressor = AutoSklearnRegressor(time_left_for_this_task=120, per_run_time_limit=30)

# Fit the model to the training data
automl_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = automl_regressor.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

In [0]:
# Define a dictionary to store the models and their corresponding RMSEs
models = {
     'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Support Vector Machine': SVR(),
    'XGBoost': XGBRegressor(),
    #'LightGBM': LGBMRegressor(),
    'CatBoost': CatBoostRegressor(logging_level='Silent'),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'MLP Neural Network': MLPRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'Elastic Net': ElasticNet()
}

best_model = None
best_rmse = float('inf')

# Loop through each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"{name} RMSE: {rmse:.2f}")
    
    # Update the best model if this model has a lower RMSE
    if rmse < best_rmse:
        best_model = model
        best_rmse = rmse

print(f"Best Model: {best_model.__class__.__name__} with RMSE: {best_rmse:.2f}")


In [0]:
# Fit the best model on the entire data (train + test)
best_model.fit(X_train, y_train)

# Get the predicted values for the entire data
y_pred = best_model.predict(X_test)
y_pred_all = best_model.predict(X)
# Create a DataFrame to store the actual and predicted values
forecast_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
forecast_df.reset_index(drop=True, inplace=True)
# Visualize the forecast
plt.figure(figsize=(10, 6))
plt.plot(forecast_df['Actual'], label='Actual', marker='o')
plt.plot(forecast_df['Predicted'], label='Predicted', marker='x')
plt.xlabel('Observation')
plt.ylabel('Total Minutes')
plt.title('Optimal Forecast vs. Actual')
plt.legend()
plt.show()

In [0]:
forecast_df

What is shap values?
A machine learning model's prediction, f(x), can be represented as the sum of its computed SHAP values, plus a fixed base value, such that:
f(x)=base_value+sum(SHAPvalues)

For instance, we see that being in the prepare advance phase have positive SHAP values (the points extending towards the right are increasingly red) and not being in the prepare advance phase have negative SHAP values (the points extending towards the left are increasingly blue). This indicates that the prepare stage is most time consuming. The reverse is seen for Heather - Heather dealt with advanving in high efficiency.

The distribution of points can also be informative. For prepare advance phase, we see a dense cluster of none prepare advance instances (blue points) with small but negative SHAP values. Instances of prepare advance (red points) extend further towards the right, suggesting being in the prepare stage has a stronger positive impact on processing time than the negative impact of not in the prepare phase on time.

The beeswarm plot is designed to display an information-dense summary of how the top features in a dataset impact the model’s output. Each instance the given explanation is represented by a single dot on each feature fow.

In [0]:
import shap
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)
# plot the distribution of importances for each feature over all samples
shap.summary_plot(shap_values,X_test,plot_type='dot',plot_size=(20,10))
feature_names = X_test.columns
shap.summary_plot(shap_values, X_test, plot_type="bar")
cb_resultX = pd.DataFrame(shap_values, columns = feature_names)

vals = np.abs(cb_resultX).values.mean(0)

shap_importance = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance.head()

In [0]:
# Create three separate DataFrames based on the conditions
df_rhi2 = X[X["Program:_rapid housing initiative 2 (rhi2)"] == True]
df_semp = X[X["Program:_shared equity mortgage program (semp)"] == True]
df_innovation = X[X["Program:_innovation fund 2"] == True]

In [0]:
import shap
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(df_rhi2)
feature_names = X_test.columns

rhi_resultX = pd.DataFrame(shap_values, columns = feature_names)

vals = np.abs(rhi_resultX).values.mean(0)
shap_importance = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['Factors','Importance'])
shap_importance.sort_values(by=['Importance'],
                               ascending=False, inplace=True)
shap_importance_filtered = shap_importance[shap_importance['Importance'] > 10]
title = "(Program:_rapid housing initiative 2 (rhi2)) Top Factors with Importance > 10"
print(title)
print('-' * len(title))  # Add a line of dashes for separation
shap_importance_filtered

In [0]:
shap_values = explainer.shap_values(df_semp)
feature_names = X_test.columns
semp_resultX = pd.DataFrame(shap_values, columns = feature_names)

vals = np.abs(semp_resultX).values.mean(0)
shap_importance = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['Factors','Importance'])
shap_importance.sort_values(by=['Importance'],
                               ascending=False, inplace=True)
shap_importance_filtered = shap_importance[shap_importance['Importance'] > 10]
title = "(Program:_shared equity mortgage program (semp)) Top Factors with Importance > 10"
print(title)
print('-' * len(title))  # Add a line of dashes for separation
shap_importance_filtered

In [0]:
shap_values = explainer.shap_values(df_innovation)
feature_names = X_test.columns
inno_resultX = pd.DataFrame(shap_values, columns = feature_names)
vals = np.abs(inno_resultX).values.mean(0)
shap_importance = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['Factors','Importance'])
shap_importance.sort_values(by=['Importance'],
                               ascending=False, inplace=True)
shap_importance_filtered = shap_importance[shap_importance['Importance'] > 10]
title = "(Program:_innovation fund 2) Top Factors with Importance > 10"
print(title)
print('-' * len(title))  # Add a line of dashes for separation
shap_importance_filtered

In [0]:
shap_importance

In [0]:
X.columns

A global feature importance plot, where the global importance of each feature is taken to be the mean absolute value for that feature over all the given samples.
Belonging to N###, Being at the ### phase or portfolio management, being dealt with ### or ####, are prone to have longer processing time.

# Simulation

Simulation involves creating a model or a representation of a real-world system or process to understand its behavior and evaluate different scenarios.
Simulations are useful when the underlying system is complex, has uncertain variables, or lacks a closed-form analytical solution.
Prediction involves estimating future outcomes or values based on historical data and patterns.

In [0]:
### Import dataset
PO = spark.table('crm_ah.msdyn_purchaseorder')

In [0]:
display(PO.select('msdyn_approvalstatus_en'))

In [0]:
FS = spark.table('crm_shared_entity.cmhc_fundingsource')

In [0]:
FS = spark.table('crm_crosslob.cmhc_application_ifarh')

In [0]:
%sql
SELECT *
FROM crm_ah.msdyn_purchaseorderproduct AS POP
INNER JOIN crm_crosslob.cmhc_application_ifarh AS CP
ON CP.id = POP.cmhc_application_ifarhid
left join crm_ah.msdyn_purchaseorder PO
on PO.id = POP.msdyn_purchaseorder;
/*refer to RITM7157542_AliastoERPGSL_viaPOP_CPandFSlevelJoiner.txt*/

In [0]:
%sql
SELECT joined_data.cmhc_programidname, COUNT(*) AS count
FROM (
    SELECT PO.cmhc_internalponumber, CP.cmhc_programidname
    FROM crm_ah.msdyn_purchaseorderproduct AS POP
    INNER JOIN crm_crosslob.cmhc_application_ifarh AS CP
    ON CP.id = POP.cmhc_application_ifarhid
    LEFT JOIN crm_ah.msdyn_purchaseorder AS PO
    ON PO.id = POP.msdyn_purchaseorder
    WHERE PO.msdyn_approvalstatus_en IS NULL
) AS joined_data
GROUP BY joined_data.cmhc_programidname;


In [0]:
selected_column = _sqldf.select('joined_data.cmhc_programidname',"count")
selected_column.show()

In [0]:
pandas_df = _sqldf.toPandas()

In [0]:
pandas_df

In [0]:
#drop1 2 4 5 6 7 32 33 34 40
rows_to_drop = [1, 2, 4, 5, 6, 7, 32, 33, 34, 40]
pandas_df_dropped = pandas_df.drop(index=rows_to_drop)


In [0]:
#keep only 12: On-Reserve Non Profit Housing Program  (Section 95)& drop rest section 95
# Assuming pandas_df is your DataFrame
rows_to_exclude = [12]
keyword = 'Section 95'

# Create a boolean mask for rows to exclude
exclude_mask = (pandas_df_dropped.index.isin(rows_to_exclude)) | (pandas_df_dropped['cmhc_programidname'].str.contains(keyword))

# Apply the mask and create a new DataFrame
pandas_df_filtered = pandas_df_dropped[~exclude_mask]

print(pandas_df_filtered)

In [0]:
folder_path = "/Workspace/Users/zzhang@cmhc-schl.gc.ca/short program name.xlsx"  # Update the folder path as per your directory structure
SN = pd.read_excel(folder_path,header=0)

In [0]:
SN

In [0]:
merged_df = pd.merge(SN, pandas_df_filtered, on='cmhc_programidname', how='inner')

In [0]:

merged_df

In [0]:
merged_df['cmhc_programidname'] = merged_df['cmhc_programidname'].apply(lambda x: x.split(']', 1)[1] if ']' in x else x)
merged_df['cmhc_programidname'] = merged_df['cmhc_programidname'].apply(lambda x: x.split('[', 1)[0] if '[' in x else x)
merged_df['cmhc_programidname'] = merged_df['cmhc_programidname'].apply(lambda x: x.split('|', 1)[0] if '|' in x else x)

In [0]:
pd.set_option('display.max_colwidth', 1000)
merged_df

In [0]:
#uniform the program name
merged_df['cmhc_programidname'] = merged_df['cmhc_programidname'].str.lower()

In [0]:
from scipy import stats
# Calculate confidence interval for predicted values
confidence_level = 0.95
degrees_freedom = len(y_test) - 1
confidence_interval = stats.t.interval(confidence_level, degrees_freedom, loc=y_pred.mean(), scale=stats.sem(y_pred))

print(f"Confidence Interval for Predicted Values: {confidence_interval}")


In [0]:
columns_starting_with_program = [col for col in TS_f.columns if col.startswith("Program")]
columns_starting_with_program

In [0]:
unique_values_count = len(columns_starting_with_program)
unique_values_count

In [0]:
# Define the columns for which you want to calculate confidence intervals
columns_to_check = columns_starting_with_program

# Calculate confidence intervals for each column where the instance equals 1
confidence_level = 0.95
degrees_freedom = len(y_test) - 1
confidence_intervals = []

for column in columns_to_check:
    indices = np.where(X[column] == True)
    pred_values = y_pred_all[indices]
    interval = stats.t.interval(confidence_level, degrees_freedom, loc=pred_values.mean(), scale=stats.sem(pred_values))
    confidence_intervals.append({'Column': column, 'Confidence Interval': interval})
    
confidence_df = pd.DataFrame(confidence_intervals)


In [0]:
unique_values_count = TS_c['Program:'].nunique()
unique_values_count 

In [0]:
confidence_df

In [0]:
confidence_df['Column'] = confidence_df['Column'].str.split('_', n=1).str.get(1)

In [0]:
merged_df

In [0]:
import pandas as pd
from fuzzywuzzy import fuzz

# Function to calculate similarity score between two strings
def calculate_similarity(str1, str2):
    return fuzz.ratio(str1, str2)

# Set the similarity threshold
similarity_threshold = 40

# Create an empty list to store the matching rows
matched_rows = []

# Iterate through each pair of values from the two DataFrames
for value_df1 in confidence_df['Column']:
    for value_df2 in merged_df['cmhc_programidname']:
        similarity_score = calculate_similarity(value_df1, value_df2)
        if similarity_score >= similarity_threshold:
            matched_rows.append({'Programs_Actual': value_df1, 'Programs_Predict': value_df2, 'similarity_score': similarity_score})

# Create a DataFrame from the list of matched rows
matched_df = pd.DataFrame(matched_rows)

# Merge the matched DataFrame with the original DataFrames
df_1 = pd.merge(matched_df, confidence_df, how='left', left_on='Programs_Actual', right_on='Column')
df_2 = pd.merge(df_1,merged_df, how='left', left_on='Programs_Predict', right_on='cmhc_programidname')

print(df_2)


In [0]:
df_2

In [0]:
df_2 = df_2.dropna(subset=['Program Short Name'])

In [0]:
# Find the index of the row with the highest similarity score for each group
idx_to_keep = df_2.groupby('Column')['similarity_score'].idxmax()

# Keep only the rows with the maximum similarity score for each group
result_df = df_2.loc[idx_to_keep]

In [0]:
unique_values_count = result_df['Column'].nunique()
unique_values_count 

In [0]:
result_df.rename(columns={'count': 'Total Number of PO'}, inplace=True)

In [0]:
result_df

In [0]:
confidence_df = result_df.iloc[:,3:]

In [0]:
result_df

In [0]:
indexes_to_replace = [7, 17,25, 37]
new_values_program_short_name = ["Indigenous Shelters", "Preservation", "RHI1", "RHI3"]
new_values_count = [7,1302,146,146]

result_df.loc[indexes_to_replace, "Program Short Name"] = new_values_program_short_name
result_df.loc[indexes_to_replace, "Total Number of PO"] = new_values_count

In [0]:
result_df

In [0]:
confidence_df = result_df.iloc[:,3:]

In [0]:
confidence_df

In [0]:
import random
for index, row in confidence_df.iterrows():
    num_values = row["Total Number of PO"]
    confidence_interval = row["Confidence Interval"]
    random_values = [random.uniform(confidence_interval[0], confidence_interval[1]) for _ in range(num_values)]
    new_column_names = [f"Random_{i}" for i in range(1, num_values + 1)]
    confidence_df.loc[index, new_column_names] = random_values

In [0]:
confidence_df

In [0]:
df_loop1 = confidence_df

In [0]:
df_loop1["Average_Random"] = df_loop1.iloc[:, 4:].mean(axis=1) 

In [0]:
df_loop1

In [0]:
# Select the desired columns
columns_to_keep = ["Column","Total Number of PO", "Confidence Interval", "Average_Random","Program Short Name"]
df_loop1 = df_loop1[columns_to_keep]

In [0]:
df_loop1

In [0]:

df_loop1.reset_index(drop=True, inplace=True)
#df_loop1.index = df_loop1.index + 1

In [0]:
df_loop1

In [0]:
num_simulations = 1000
average_values = []

for index, row in df_loop1.iterrows():
    num_values = row["Total Number of PO"]
    confidence_interval = row["Confidence Interval"]
    averages = []
    
    for _ in range(num_simulations):
        random_values = [random.uniform(confidence_interval[0], confidence_interval[1]) for _ in range(num_values)]
        average_random = sum(random_values) / num_values
        averages.append(average_random)
    
    average_values.append(averages)

# Convert average_values to a DataFrame
average_df = pd.DataFrame(average_values)
average_df.columns = [f"Average_{i+1}" for i in range(num_simulations)]

# Concatenate average_df with the original DataFrame
result_df = pd.concat([df_loop1, average_df], axis=1)

In [0]:
result_df

In [0]:
# Generate random values within the interval and calculate averages
num_simulations = 1000
average_values = []

for index, row in confidence_df.iterrows():
    num_values = row["Total Number of PO"]
    confidence_interval = row["Confidence Interval"]
    averages = []
    
    for _ in range(num_simulations):
        random_values = [random.uniform(confidence_interval[0], confidence_interval[1]) for _ in range(num_values)]
        average_random = sum(random_values) / num_values
        averages.append(average_random)
    
    average_values.append(averages)

# Convert average_values to a DataFrame
average_df = pd.DataFrame(average_values)
average_df.columns = [f"Average_{i+1}" for i in range(num_simulations)]

# Calculate confidence interval and mean for each row
confidence_intervals = []
means = []

for index, row in average_df.iterrows():
    data = row.values
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)
    margin_of_error = stats.t.ppf((1 + 0.95) / 2, df=num_simulations - 1) * (std_dev / np.sqrt(num_simulations))
    confidence_interval = (mean - margin_of_error, mean + margin_of_error)
    means.append(mean)
    confidence_intervals.append(confidence_interval)

# Add mean and confidence interval columns to the result_df
result_df["Mean(mins)"] = means
result_df["Confidence_Interval_final"] = confidence_intervals


In [0]:
result_df

In [0]:
columns_to_keep = ["Column",'Program Short Name',"Total Number of PO", "Confidence_Interval_final","Mean(mins)"]
df = result_df[columns_to_keep]

In [0]:
df

In [0]:
df["Total Processing Time(hrs)"] = df["Total Number of PO"] * df["Mean(mins)"] / 60

In [0]:
df

In [0]:
df['Confidence_Interval_final'] = df['Confidence_Interval_final'].apply(lambda x: (round(x[0], 2), round(x[1], 2)))

In [0]:
df.rename(columns={
    'Column': 'Program',
    'Total Number of PO': 'Total # of Unapproved PO of Each Program',
    'Confidence_Interval_final': 'Confidence Interval of Average Processing Time of Each PO(Minutes)',
    'Mean(mins)':"Average Processing Time of Each PO(Minutes)",
    "Total Processing Time(hrs)":"Total Processing Time of Each Program(Hours)"
}, inplace=True)


In [0]:
df

In [0]:
df

In [0]:
# Drop the specified column
df.drop('Confidence Interval of Average Processing Time of Each PO(Minutes)', axis=1, inplace=True)

In [0]:
pd.DataFrame.iteritems = pd.DataFrame.items
from pyspark.sql import SparkSession
# Create a Spark session
spark = SparkSession.builder.getOrCreate()
# Assuming 'pandas_df' is your Pandas DataFrame
spark_df = spark.createDataFrame(df)
display(spark_df)